In [9]:
import tkinter as tk
from tkinter import messagebox
from tkinter import simpledialog

In [10]:
class SortedBinaryNode:
    indent = '  '
    node_radius = 10    # Radius of a node's circle
    x_spacing = 20      # Horizontal distance between neighbouring subtrees
    y_spacing = 20      # Vertical distance between parent and child subtrees
    
    def __init__(self, value=''):
        self.value = value
        self.left_child = None
        self.right_child = None
        
        # Initialise drawing parameters
        self.centre = (0, 0)
        self.subtree_bounds = (
            self.centre[0] - self.node_radius,
            self.centre[1] - self.node_radius,
            self.centre[0] + self.node_radius,
            self.centre[1] + self.node_radius
        )

    def __str__(self, level=0):
        return (
            f"{self._indent_node(self.value, level=level)}:\n"
            f"{self.left_child.__str__(level=level+1) if self.left_child else self._indent_missing_child_node('None', level=level+1)}"
            f"{self.right_child.__str__(level=level+1) if self.right_child else self._indent_missing_child_node('None', level=level+1)}"
        )

    def _indent_missing_child_node(self, value='', level=0):
        """
        Prepares a string indented to reflect the level of the empty child of the current node. 
        :param value: the text, if available, to include after the indentation 
        :param level: an integer to reflect the level (or depth) in the tree and the indentation required for the node
        :return: string containing enough indentation chars (usually spaces) followed by a text value if available
        """
        return self._indent_node(value + '\n', level=level) if self.left_child or self.right_child else ''

    @classmethod
    def _indent_node(cls, value='', level=0):
        """
        Prepares a string indented to reflect the level of the current node in the tree. 
        :param value: the text, if available, to include after the indentation
        :param level: an integer to reflect the level (or depth) in the tree and the indentation required for the node
        :return: string containing enough indentation chars (usually spaces) followed by a text value if available
        """
        return "".join([cls.indent for _ in range(0, level)]) + value

    def add_node(self, child):
        # Check if new child's value is smaller than this node
        if child.value < self.value:
            # Add it to the left subtree
            if self.left_child:
                self.left_child.add_node(child)
            else:
                self.left_child = child
        elif child.value > self.value:
            # Add it to the right subtree
            if self.right_child:
                self.right_child.add_node(child)
            else:
                self.right_child = child
        else:
            # Duplicate value
            raise ValueError(f"Cannot add duplicate value: {child.value}")
        
    def find_node(self, value):
        """Recursively search this node's subtree looking for the target value.
        Return the node that contains the value or None."""
        if self.value == value:
            return self
        elif value < self.value:
            return self.left_child.find_node(value) if self.left_child else None
        else:
            return self.right_child.find_node(value) if self.right_child else None

    def traverse_preorder(self):
        nodes = [self]
        if self.left_child:
            nodes.extend(self.left_child.traverse_preorder())
        if self.right_child:
            nodes.extend(self.right_child.traverse_preorder())
        return nodes
    
    def traverse_inorder(self):
        nodes = self.left_child.traverse_inorder() if self.left_child else []
        nodes.extend([self])
        if self.right_child:
            nodes.extend(self.right_child.traverse_inorder())
        return nodes
    
    def traverse_postorder(self):
        nodes = self.left_child.traverse_postorder() if self.left_child else []
        if self.right_child:
            nodes.extend(self.right_child.traverse_postorder())
        nodes.extend([self])
        return nodes
    
    def traverse_breadth_first(self):
        result = []
        queue = [self]
        
        while queue:
            node = queue.pop(0)
            result.append(node)
            
            for c in [node.left_child, node.right_child]:
                queue.append(c) if c else None
        
        return result

    def arrange_subtree(self, xmin, ymin):
        """Position the node's subtree"""
        # Set ymax to the bottom of this node
        ymax = ymin + 2 * self.node_radius
        xmax = xmin
        
        # See if the node has any children
        if not self.left_child and not self.right_child:
            # There are no children. Put the node here.
            xmax += 2 * self.node_radius
            self.subtree_bounds = (xmin, ymin, xmax, ymax)
        else:
            ymax += self.y_spacing
            
            if self.left_child:
                self.left_child.arrange_subtree(xmax, ymax)
                
                # Update xmax to allow room for the next subtree
                xmax = self.left_child.subtree_bounds[2]
                
                # If there is also a right child, allow room between them
                if self.right_child:
                    xmax += self.x_spacing
                
            # Position the right subtree
            if self.right_child:
                self.right_child.arrange_subtree(xmax, ymax)
                
                # Update xmax
                xmax = self.right_child.subtree_bounds[2]
    
        # Position the node
        if not self.left_child and not self.right_child:
            cx = (self.subtree_bounds[0] + self.subtree_bounds[2]) / 2
        elif not self.left_child:
            cx = self.right_child.centre[0]
            ymax = self.right_child.subtree_bounds[3]
        elif not self.right_child:
            cx = self.left_child.centre[0]
            ymax = self.left_child.subtree_bounds[3]
        else:
            cx = (self.left_child.centre[0] + self.right_child.centre[0]) / 2
            ymax = max(self.left_child.subtree_bounds[3], self.right_child.subtree_bounds[3])
        self.subtree_bounds = (xmin, ymin, xmax, ymax)
        
        cy = ymin + self.node_radius
        self.centre = (cx, cy)
    
    def draw_subtree_links(self, canvas):
        """Draw the subtree's links"""
        if self.left_child:
            self.left_child.draw_subtree_links(canvas)
            canvas.create_line(
                self.centre, 
                self.left_child.centre,
                fill='black'
            )
        if self.right_child:
            self.right_child.draw_subtree_links(canvas)
            canvas.create_line(
                self.centre, 
                self.right_child.centre,
                fill='black'
            )
            
        # Outline the subtree for debugging
        #canvas.create_rectangle(self.subtree_bounds, fill='', outline='red')
        
    def draw_subtree_nodes(self, canvas):
        """Draw the subtree's nodes"""
        # Draw the node
        canvas.create_oval(
            self.centre[0] - self.node_radius, 
            self.centre[1] - self.node_radius, 
            self.centre[0] + self.node_radius, 
            self.centre[1] + self.node_radius, 
            fill='white',
            outline='green'
        )
        canvas.create_text(self.centre, text=self.value, fill='red')
        
        # Draw the descendant's nodes
        if self.left_child:
            self.left_child.draw_subtree_nodes(canvas)
        if self.right_child:
            self.right_child.draw_subtree_nodes(canvas)
        
    def arrange_and_draw_subtree(self, canvas, xmin, ymin):
        # Position the tree
        self.arrange_subtree(xmin, ymin)
        
        # Draw the links
        self.draw_subtree_links(canvas)
        
        # Draw the nodes
        self.draw_subtree_nodes(canvas)

    def pop(self, target):
        """
        Find the node that contains this value, remove it from the subtree, and return it.
        
        When this method starts, we know that this node is not the target node.
        Search the child subtrees.
        """
        return self._delete(target)
    
    def _replace_subtree(self, child, subtree):
        """Replaces the child for the new subtree"""
        if self.left_child == child:
            self.left_child = subtree
        elif self.right_child == child:
            self.right_child = subtree
    
    def _delete(self, target, subtree=None):
        if target < self.value:
            return self.left_child._delete(target, self) if self.left_child else None
        elif target > self.value:
            return self.right_child._delete(target, self) if self.right_child else None
        # this node is the target for removal
        elif not self.left_child and not self.right_child:
            subtree._replace_subtree(self, None) if subtree else None
        elif not self.left_child:
            subtree._replace_subtree(self, self.right_child) if subtree else None
        elif not self.right_child:
            subtree._replace_subtree(self, self.left_child) if subtree else None
        else:
            # both children are present
            self.value = self.left_child._delete_max(self)
        return self
    
    def _delete_max(self, subtree):
        """Returns the value of and removes the highest-valued node in the subtree"""
        if not self.right_child:
            # this node is the highest-valued
            subtree._replace_subtree(self, self.left_child)
            return self.value
        else:
            # the node has a right child
            return self.right_child._delete_max(self)


In [11]:
class App:
    def __init__(self):
        # Make a sentinel root
        self.root = SortedBinaryNode(-1)
        self.run_tests()

        # Make tkinker window
        self.window = tk.Tk()
        self.window.title('SortedBinaryNode')
        self.window.protocol('WM_DELETE_WINDOW', self.kill_callback)
        self.window.geometry('290x320')
        
        frame = tk.Frame(self.window)
        frame.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

        self.value_entry = tk.Entry(frame, width=3)
        self.value_entry.pack(padx=(10, 0), side='left')
        self.value_entry.focus_set()
        
        self.add_button = tk.Button(frame, text='Add', width=8, command=self.add_value)
        self.add_button.pack(padx=(10, 0), side='left')

        self.pop_button = tk.Button(frame, text='Pop', width=8, command=self.pop_value)
        self.pop_button.pack(padx=(10, 0), side='left')

        self.find_button = tk.Button(frame, text='Find', width=8, command=self.find_value)
        self.find_button.pack(padx=(10, 0), side='left')
        
        self.canvas = tk.Canvas(self.window, bg='white', borderwidth=2, relief=tk.SUNKEN)
        self.canvas.pack(padx=10, pady=(0, 10), fill=tk.BOTH, expand=True)
        
        # Make some shortcuts
        self.window.bind_all('<Control-a>', self.ctrl_a_pressed)
        self.window.bind_all('<Control-p>', self.ctrl_p_pressed)
        self.window.bind_all('<Control-f>', self.ctrl_f_pressed)
        
        # Draw the initial tree
        self.draw_tree()
        
        self.window.focus_force()
        self.window.mainloop()
        
    def kill_callback(self):
        """A callback to destroy the tkinter window"""
        self.window.destroy()
        
    def draw_tree(self):
        # Remove any previous widgets
        self.canvas.delete('all')
        if self.root:
            self.root.arrange_and_draw_subtree(self.canvas, 10, 10)
    
    def ctrl_a_pressed(self, event):
        self.add_value()
    def add_value(self):
        # Add a value to the tree
        new_string = self.value_entry.get()
        if not new_string:
            return
        
        self.value_entry.delete(0, 'end')
        self.value_entry.focus_set()
        
        try:
            new_value = int(new_string)
        except Exception as e:
            messagebox.showinfo('Add Error',
                                f"Value {new_string} must be an integer.\n{e}")
            return
        
        if new_value <= 0:
            messagebox.showinfo('Add Error',
                                f"Value {new_value} must be a positive integer.")
            return
        
        try:
            new_node = SortedBinaryNode(new_value)
            self.root.add_node(new_node)
        except Exception as e:
            messagebox.showinfo('Add Error',
                                f"Error adding value {new_value} to the tree.\n{e}")
        
        self.draw_tree()
        
    def ctrl_p_pressed(self, event):
        self.pop_value()
    def pop_value(self):
        # Remove a value from the tree
        target_string = self.value_entry.get()
        if not target_string:
            return
        
        self.value_entry.delete(0, 'end')
        self.value_entry.focus_set()
        
        try:
            target = int(target_string)
        except Exception as e:
            messagebox.showinfo('Pop Error',
                                f"Value {target_string} must be an integer.\n{e}")
            return
        
        try:
            node = self.root.pop(target)
            if not node:
                messagebox.showinfo('Value Error',
                                    f"The value {target} is not in the tree.")
        except Exception as e:
            messagebox.showinfo('Pop Error',
                                f"Error removing value {target} from the tree.\n{e}")
            
        self.draw_tree()
        
    def ctrl_f_pressed(self, event):
        self.find_value()
    def find_value(self):
        # Find the value's node
        target_string = self.value_entry.get()
        if not target_string:
            return
        
        self.value_entry.delete(0, 'end')
        self.value_entry.focus_set()
        
        try:
            target = int(target_string)
        except Exception as e:
            messagebox.showinfo('Find Error',
                                f"Value {target_string} must be an integer.\n{e}")
            return
        
        try:
            node = self.root.find_node(target)
            if not node:
                messagebox.showinfo('Not found',
                                    f"The value {target} is not in the tree.")
            else:
                messagebox.showinfo('value Found',
                                    f"Found node with value {node.value}.")
        except Exception as e:
            messagebox.showinfo('Find Error',
                                f"Error finding value {target} in the tree.\n{e}")
            
        # Redraw the tree
        self.draw_tree()
    
    def run_tests(self):
        self.root.add_node(SortedBinaryNode(60))
        self.root.add_node(SortedBinaryNode(35))
        self.root.add_node(SortedBinaryNode(76))
        self.root.add_node(SortedBinaryNode(21))
        self.root.add_node(SortedBinaryNode(42))
        self.root.add_node(SortedBinaryNode(71))
        self.root.add_node(SortedBinaryNode(89))
        self.root.add_node(SortedBinaryNode(17))
        self.root.add_node(SortedBinaryNode(24))
        self.root.add_node(SortedBinaryNode(74))
        self.root.add_node(SortedBinaryNode(11))
        self.root.add_node(SortedBinaryNode(23))
        self.root.add_node(SortedBinaryNode(72))
        self.root.add_node(SortedBinaryNode(75))
        
        # test find_node for each of the nodes added above
        result = [(n.value, 'Found in the tree' if self.root.find_node(n.value) else None) for n in self.root.traverse_preorder()]
        nodes_not_found = list(filter(lambda n: not n[1], result))
        if nodes_not_found:
            messagebox.showinfo('Test Find Nodes', f"Failed to find nodes: {[n[0] for n in nodes_not_found]}")
        

In [12]:
App()

# Tests:
# Add values in order: O, G, S, C, I, A, U, M, Q, E. Verify that you get this tree:
#          K
#         / \
#        /   \
#       /     \
#      G       O
#     / \     / \
#    /   \   /   \
#   C     I M     S
#  / \           / \ 
# A   E         Q   U
#
# Find each value and verify that it is in the tree.
# Find values H, f, X, 7 and verify that they are not in the tree.
# Try to add values C, E, M, and K again and verify that you get errors.
